In [ ]:
import time
from flask import Flask, request, jsonify
import requests
import google.generativeai as genai
import google.logging as logging
import json
import os
from flask_cors import CORS
import dotenv
import datetime

dotenv.load_dotenv()

AMADEUS_CLIENT_ID = os.getenv("AMADEUS_CLIENT_ID")
AMADEUS_CLIENT_SECRET = os.getenv("AMADEUS_CLIENT_SECRET")

NEW_GOOGLE_API_KEY = os.getenv("NEW_GOOGLE_API_KEY")
genai.configure(api_key=NEW_GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')
# model = genai.GenerativeModel('gemini-1.5-pro')



chat = model.start_chat(
    history=[]
)

def generate_travel_guide():
    data = request.get_json()
    user_input = data.get('user_input', '')
    print(f"User input: {user_input}")
    
    obj = {
        "responseText": "Your response here...",
        "details": [
            {"parameter": "departure", "value": "<value>"},
            {"parameter": "arrival", "value": "<value>"},
            {"parameter": "start_date", "value": "<value>"},
            {"parameter": "end_date", "value": "<value>"},
            {"parameter": "numAdults", "value": "<value>"},
            {"parameter": "numChildren", "value": "<value>"},
            {"parameter": "numInfants", "value": "<value>"},
            {"parameter": "isOneWay", "value": "<true/false>"},
            {"parameter": "budget", "value": "<value>"}
        ]
    }

    prompt = f"""   
        The user is going to give you what kind of flight you need to find for them. 
        Your job is to update the details, so do that. Use the information the user is 
        giving you to do that. Do not add any extra text besides just filling in the values. No text before. No text after. We want 
        to ensure there are no parsing errors so this step is important. If the user is talking about where they would like to travel to, please use the input to fill the 'arrival' parameter.
        If the user is asking about the dates, please use the input to fill the 'start_date' and 'end_date' parameters.
        If the user is asking about the number of people traveling, please use the input to fill the 'numAdults', 'numChildren', and 'numInfants' parameters.
        to do that Remember do not include formatting or code blocks!!!!!!!!!!
        SUPER IMPORTANT.
        here is the users query: "{user_input}".

        Please also put in the responseText field, a comment to the user, and if you need any more details, please add them here.
    {json.dumps(obj)}
    """

    # try:
    response = chat.send_message(prompt)
    # ✅
    response_text = response.text
    print(f"response_text: {response_text}")
    print()
    # ✅
    clean_response_text = response_text.replace("```json\n", "").replace("\n```", "")
    print(f"clean_response_text: {clean_response_text}")
    print()

    # ✅ 
    response_json = json.loads(clean_response_text)
    print(f"response_json: {response_json}")
    print()
    
    # Extract flight search details
    flight_search_details = response_json.get('details', [])
    print(f"flight_search_details: {flight_search_details}")
    print()
    search_params = {param['parameter']: param['value'] for param in flight_search_details}
    print(f"search_params: {search_params}")
    print()
    
    # Prepare parameters for flight search
    departure = search_params.get('departure')
    arrival = search_params.get('arrival')
    start_date = search_params.get('start_date')
    end_date = search_params.get('end_date')
    num_adults = search_params.get('numAdults')
    num_children = search_params.get('numChildren')
    is_one_way = search_params.get('isOneWay', 'true') == 'true'

    print()
    print('Departure:', departure)
    print('Arrival:', arrival)
    print('Start date:', start_date)
    print('End date:', end_date)

    # except Exception as e:
    #     print(f"Error generating search parameters: {e}")
    #     return jsonify({'error': f'An error occurred: {str(e)}'}), 500

    try:
        # Ensure that required parameters are present
        # if departure and arrival and start_date and end_date:
        if departure and arrival and start_date:
            access_token = get_access_token()
            if not access_token:
                return jsonify({'error': 'Unable to fetch access token'}), 500

            # Convert parameters to integers if present and not 'unknown'
            num_adults = int(num_adults) if num_adults and num_adults != 'unknown' else 1
            num_children = int(num_children) if num_children and num_children != 'unknown' else 0

            flight_data = search_flights(
                access_token,
                city_from=departure if not is_one_way else departure,
                city_to=arrival if not is_one_way else arrival,
                departure_date=start_date if start_date else None,
                return_date=end_date if end_date else None,
                adults=num_adults if num_adults else 1,
                children=num_children if num_children else None,
                is_one_way=is_one_way
            )
            response_json['searched_flights'] = flight_data if flight_data else {}
        else:
            response_json['searched_flights'] = {}

        return jsonify(response_json)
    
    except Exception as e:
        return jsonify({'error': f'An error occurred during flight search: {str(e)}'}), 500
